# One Axes Figure

This example demonstrates how to create a one axes figure with fixed margins around the axes using both `mpllayout` and pure `matplotlib`.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from mpllayout import primitives as pr
from mpllayout import constraints as co
from mpllayout import layout as lay
from mpllayout import solver
from mpllayout import matplotlibutils as lplt


## Creating the layout using `mpllayout`

In [ ]:
layout = lay.Layout()

## Create the figure and axes
layout.add_prim(pr.Quadrilateral(), "Figure")
layout.add_prim(pr.Axes(), "Axes")

## Constrain figure and axes quadirlateral to be rectangular
layout.add_constraint(co.Box(), ("Figure",), ())
layout.add_constraint(co.Box(), ("Axes/Frame",), ())

## Constrain the figure size
fig_width, fig_height = 6, 3
layout.add_constraint(co.XLength(), ("Figure/Line0",), (fig_width,))
layout.add_constraint(co.YLength(), ("Figure/Line1",), (fig_height,))

## Constrain 'Axes' margins
# Constrain left/right margins
margin_left = 1.1
margin_right = 1.1
layout.add_constraint(
    co.InnerMargin(side='left'), ("Axes/Frame", "Figure"), (margin_left,)
)
layout.add_constraint(
    co.InnerMargin(side='right'), ("Axes/Frame", "Figure"), (margin_right,)
)

# Constrain top/bottom margins
margin_top = 1.1
margin_bottom = 0.5
layout.add_constraint(
    co.InnerMargin(side='bottom'), ("Axes/Frame", "Figure"), (margin_bottom,)
)
layout.add_constraint(
    co.InnerMargin(side='top'), ("Axes/Frame", "Figure"), (margin_top,)
)

## Solve the constraints
prim_tree_n, info = solver.solve(layout)

In [ ]:
# Used the solved constraints to form the figure and axes objects, then plot
fig, axs = lplt.subplots(prim_tree_n)

ax = axs["Axes"]

x = np.linspace(0, 1)
ax.plot(x, x**2)

ax.set_xlabel("x")
ax.set_ylabel("y")

## Creating the layout using pure `matplotlib`

There are multiple approaches to create the above layout using pure `matplotlib`; however, some may not have the precise margins specified.
To create the layout using pure `matplotlib`, the axes position that satisfies the desired margins and figure size must be determined.
This essentially involves manually "solving" the system of constraints.
For the simple case of fixed margins around a single axes, the solution is given by the code below.
This is the basic process that `mpllayout` performs but for general constraints, which allows for more complicated layouts.

In [ ]:
## This code determines the axes sizes needed to achieve the desired margins

# Specify the desired figure size and margins
fig_width, fig_height = 6, 3

margin_left = 1.1
margin_right = 1.1

margin_top = 1.1
margin_bottom = 0.5

# The code below "solves" the axes position needed to achieve the given margins.
# Using this approach calculates maintains the same margins even if the figure
# dimensions are changed.
# This is essentially what `mpllayout` does.
coord_botleft = np.array((margin_left, margin_bottom))
coord_topright = np.array((fig_width - margin_right, fig_height-margin_top))

# Scale the coordinates relative to the figure size since this is how `matplotlib`
# interprets the axes position
coord_botleft = coord_botleft / (fig_width, fig_height)
coord_topright = coord_topright / (fig_width, fig_height)

# Determine the axes width and height
axes_width, axes_height = (coord_topright - coord_botleft)

In [ ]:
# Use the "solved" axes position and figure size to create the figure
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes((*coord_botleft, axes_width, axes_height))

x = np.linspace(0, 1)
ax.plot(x, x**2)

ax.set_xlabel("x")
ax.set_ylabel("y")

# fig